In [ ]:
from __future__ import print_function
import os
import numpy as np
import scipy.stats as sista
import scipy.io as sio
import mat73 as mt
import matplotlib.pyplot as plt
#import mord

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from statsmodels.stats.multitest import multipletests

np.random.seed(42)

In [ ]:
#project directory
cwd = os.getcwd()

#subjects to include
subj_name = ['1','2','4','5','6','7','8','9','11','12']
nsubj = len(subj_name)

#number of electrodes
ne = 30

#trial timing
t0 = -400
te = 1551

# Classification parameters and setup
time_window = 50 #bin width
time_step = 25 #bin offset (if smaller than tw, will have overlap)
n_splits = 1000 #no of iterations
trial_average = [20] #no of trials to average

# ss: 2 vs 4; cond: same vs diff vs simultaneous
group_label = ['condition','setSize'] #conditions to be referenced
group_dict = {0:[2,2],1:[1,4]}

cs = group_dict
num_cs = len(group_dict)

# create experiment, data wrangler, and classification objects 
samples = np.arange(t0,te,2)
sample_step = samples[1]-samples[0]
t = samples[0:samples.shape[0]-int(time_window/sample_step)+1:int(time_step/sample_step)]

#train/test indices using stratified randomized folds
#preserves the percentage of samples for each class
cross_val = StratifiedShuffleSplit(n_splits,test_size=0.2)
classifier = LogisticRegression()
#z-score normalization
scaler = StandardScaler()
le = LabelEncoder()

#load labels
fn = cwd + '/allsubs_data.mat'
beh = mt.loadmat(fn)

#set up directories
df = cwd + '/EEGData/'

In [ ]:
#preallocate
acc = np.zeros((nsubj,np.size(t),n_splits))*np.nan
acc_shuff = np.zeros((nsubj,np.size(t),n_splits))*np.nan
conf_mat = np.zeros((nsubj,np.size(t),n_splits,num_cs,num_cs))*np.nan

In [ ]:
# loop over subjects in experiment
for isubj in range(nsubj):
    print(isubj)

    #filename
    fn = df + subj_name[isubj] +'_EEG.mat'

    subj_mat = mt.loadmat(fn,only_include = ['erp/arfDat/baselined','erp/arf/artifactIndCleaned'])
    
    #artifact rejection indices, use only clean trials
    idx = subj_mat['erp']['arf']['artifactIndCleaned']==0
    
    #data, baselined ERP
    erp_baselined = subj_mat['erp']['arfDat']['baselined']
    erp_baselined = erp_baselined[idx,:ne,:]

    #condition labels
    labels = np.zeros((len(group_label),len(idx)))*np.nan
    for icond,cond in enumerate(group_label):
        labels[icond,:] = beh['allsubs'][group_label[icond]][isubj][:]

    cnt = 0
    #conditions of interest, specified above
    values = list(group_dict.values())
    #preallocate to save new labels that only includes conditions of interest
    labels_new = np.zeros((len(idx)))*np.nan
    for val in values:
        #for this type of trial, preallocate combinations of variables
        include = np.zeros((len(group_label),len(idx)))*np.nan
        for icond, cond in enumerate(group_label):
            include[icond,:] = np.isin(labels[icond,:],val[icond])
        # include trial if all conditions are satisfied
        labels_new[np.sum(include,axis=0)==len(group_label)] = cnt
        cnt += 1
        
    labels = labels_new
    labels = np.array(labels)
    labels = labels[idx] 

    #only trials from conditions of interest
    label_idx = ~np.isnan(labels)
    xdata = erp_baselined[label_idx,:,:]
    ydata = labels[label_idx]
    
    le.fit(ydata)
    ydata = le.fit_transform(ydata)
    
    #balance the number for each class
    unique_labels, counts_labels = np.unique(ydata, return_counts=True)
    downsamp = min(counts_labels)
    label_idx=[]
    for label in unique_labels:
        label_idx = np.append(label_idx,np.random.choice(np.arange(len(ydata))[ydata == label],downsamp,replace=False))
    #include only selected trials
    xdata = xdata[label_idx.astype(int)]
    ydata = ydata[label_idx.astype(int)]

    #why was balancing happening before this
    #average trials (binning)
    #already equating number of trials here?
    unique_labels, counts_labels = np.unique(ydata, return_counts=True)
    min_count = np.floor(min(counts_labels)/trial_average)*trial_average
    nbin = int(min_count/trial_average) #how many mega-trials per condition
    trial_groups = np.tile(np.arange(nbin),trial_average)
    #mega_trials X electrodes X time
    xdata_new = np.zeros((nbin*len(unique_labels),xdata.shape[1],xdata.shape[2]))
    count = 0
    #for each mega-trial of each condition
    for ilabel in unique_labels:
        for igroup in np.unique(trial_groups):
            #find trials in this condition, grab the first min_count, average across their groups
            #always choosing the first trials and cutting off the last
            #same trials are getting averaged over??
            xdata_new[count] = np.mean(xdata[ydata==ilabel][:int(min_count)][trial_groups==igroup],axis=0)
            count += 1
            
    #run classification on averaged data        
    ydata_new = np.repeat(unique_labels,nbin)
    xdata,ydata = xdata_new,ydata_new

    ifold = 0

    #split data into train and test, training samples are not balanced but close
    for train_index, test_index in cross_val.split(xdata[:,0,0],ydata):
        
        #randomizing which groups of averaged trials will serve as train and test
        X_train_all, X_test_all = xdata[train_index], xdata[test_index]

        #don't see an imbalance issue
        y_train, y_test = ydata[train_index].astype(int), ydata[test_index].astype(int)
        if sum(y_test==1)==sum(y_test==0)== False:
            k

        #shuffle labels for chance classification
        y_test_shuffle = np.random.permutation(y_test)

        #for each time point
        for itime, time in enumerate(t):

            time_window_idx = (samples >= time) & (samples <  time + time_window)

            #data for this time bin
            X_train = np.mean(X_train_all[...,time_window_idx],2)
            X_test = np.mean(X_test_all[...,time_window_idx],2)

            #z-score each electrode across trials at this time point
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            
            #do classification
            classifier.fit(X_train, y_train)

            acc[isubj,itime,ifold] = classifier.score(X_test,y_test)
            acc_shuff[isubj,itime,ifold] = classifier.score(X_test,y_test_shuffle)
            conf_mat[isubj,itime,ifold] = confusion_matrix(y_test,y_pred=classifier.predict(X_test))

        ifold += 1


In [ ]:
sio.savemat('ss2d_ss4s_exp1.mat', mdict={'acc': acc, 'acc_shuff': acc_shuff,'conf_mat':conf_mat})